In [13]:
import tensorflow as tf
import numpy as np
import models
import helpers
import data_utils

### This Notebook is to vrify whether model is really memorizing examples i.e. whether it is good model to start with

In [14]:
metadata, idx_q, idx_a = data_utils.ourmodel.data_util.load_data()
(trainX, trainY), (testX, testY), (validX, validY) = helpers.split_dataset(idx_q, idx_a)

In [15]:

# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 32


In [16]:
xvocab_size

20002

In [17]:
trainXX = trainX[1:200]
trainYY = trainY[1:200]

In [18]:
reload(models.seq2seq_wrapper)
from models import seq2seq_wrapper
model = seq2seq_wrapper.Seq2Seq(xseq_len= xseq_len,
                               yseq_len= yseq_len,
                               xvocab_size= xvocab_size,
                               yvocab_size= yvocab_size,
                               ckpt_path='ckpt/checkpoint/',
                               emb_dim= emb_dim,
                               num_layers=2,
                             epochs = 4000,
                                lr = 0.005,
                                attention = False,
                                celltype = 'GRU'
                               )



<log> Building Graph <log> Building Input variables of Graph <log> Cells <log> Building Actual model .AdamOptimizer.. </log>

In [19]:
batch_size = 16

val_batch_gen = helpers.rand_batch_gen(validX, validY, 2)
train_batch_gen = helpers.rand_batch_gen(trainXX, trainYY, batch_size)

### Execute Training if model is not available from prior runs

In [ ]:


sess = model.train(train_batch_gen, val_batch_gen)

In [20]:

sess = model.restore_last_session()

In [21]:
input_,output_ = train_batch_gen.next()
output = model.predict(sess, input_)
print(output.shape)

(16, 20)


In [22]:
replies = []

for ii, oi,ot in zip(input_.T, output,output_.T):
    q = helpers.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = helpers.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    rs = helpers.decode(sequence=ot, lookup=metadata['idx2w'], separator=' ').split(' ')
    #if decoded.count('unk') == 0:
     #   if decoded not in replies:
    print('Review : [{0}]; Summary : [{1}];real summary :[{2}] '.format(q, ' '.join(decoded),' '.join(rs)))
    #print "Real Summary %s",(helpers.decode(sequence=ot, lookup=metadata['idx2w'], separator=' ').split(' '))
    replies.append(decoded)

Review : [watch prices assortment good get gold box purchase price unk 34 less target]; Summary : [better price target];real summary :[better price target] 
Review : [confection around centuries light unk citrus gelatin nuts case unk cut tiny squares liberally coated powdered sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story unk unk]; Summary : [delight says];real summary :[delight says] 
Review : [cat food recommended vet 13 year old unk cat trouble keeping food loosing weight stopped problems doesnt eat much since seems satisfied eating helps justify added price aids digestion since]; Summary : [perfect cat food older cats];real summary :[perfect cat food older cats] 
Review : [dont know long sat back shelf somewhere old wouldnt cook throw half skins damaged clear sign dried beans past prime cant even return ask refund food returnable find different brand]; Summary : [low quality];real summary :[low quality] 
Review : [keep trying brands cheaper b